In [1]:
from tensorflow import keras
import os
import joblib
import numpy as np
from explain import  get_relevance, get_critical_neurons
import tensorflow as tf
# from tensorflow import set_random_seed
from scalelayer import  ScaleLayer
from numpy.random import seed
import itertools
import time
import copy
from preprocessing import pre_census_income
import tensorflow.keras.backend as KTF
import argparse

seed(1)
tf.random.set_random_seed(2)


def similar_set(X, num_attribs, protected_attribs, constraint):
    # find all similar inputs corresponding to different combinations of protected attributes with non-protected attributes unchanged
    similar_X = []
    protected_domain = []
    for i in protected_attribs:
        protected_domain = protected_domain + [list(range(constraint[i][0], constraint[i][1]+1))]
    all_combs = np.array(list(itertools.product(*protected_domain)))
    for i, comb in enumerate(all_combs):
        X_new = copy.deepcopy(X)
        for a, c in zip(protected_attribs, comb):
            X_new[:, a] = c
        similar_X.append(X_new)
    return similar_X



In [2]:
X_train, X_val, y_train, y_val, constraint = pre_census_income.X_train, \
    pre_census_income.X_val, pre_census_income.y_train, pre_census_income.y_val, pre_census_income.constraint
    
X_test, y_test = pre_census_income.X_test, pre_census_income.y_test

pos_map = { 'a': [0],
            'r': [6],
            'g': [7],
            'a&r': [0, 6],
            'a&g': [0, 7],
            'r&g': [6, 7]
            }

models_map = { 'a': "models/retrained_adv/a_adult_model_0.h5",
            'r': "models/retrained_adv/r_adult_model_0.h5",
            'g': "models/retrained_adv/g_adult_model_0.h5",
            'a&r': "models/retrained_adv/a&r_adult_model_0.h5",
            'a&g': "models/retrained_adv/a&g_adult_model_0.h5",
            'r&g': "models/retrained_adv/r&g_adult_model_0.h5",
    
}

for attr in pos_map.keys():
    print("attr", attr)
    target_model_path = models_map[attr]
    data_name = f"discriminatory_data/adult/C-{attr}_ids_EIDIG_INF.npy"
    dis_data = np.load(data_name)
    num_attribs = len(X_train[0])
    protected_attribs = pos_map[attr]
    similar_X = similar_set(dis_data, num_attribs, protected_attribs, constraint)

    augmented_model = keras.models.load_model(target_model_path)
    aug_data = (augmented_model.predict(dis_data) > 0.5).astype(int).flatten()
    dis_num = 0
    newdata_res = []

    l = len(similar_X)
    for i in range(l):
        newdata_re = augmented_model.predict(similar_X[i])
        newdata_re = (newdata_re > 0.5).astype(int).flatten()
        newdata_res.append(newdata_re)
    repaired_num = get_repaired_num(newdata_res)

    print('Aug', repaired_num/len(dis_data))


attr a
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


NameError: name 'get_repaired_num' is not defined

In [ ]:
X_train, X_val, y_train, y_val, constraint = pre_census_income.X_train, \
    pre_census_income.X_val, pre_census_income.y_train, pre_census_income.y_val, pre_census_income.constraint
    
X_test, y_test = pre_census_income.X_test, pre_census_income.y_test

pos_map = { 'a': [0],
            'r': [6],
            'g': [7],
            'a&r': [0, 6],
            'a&g': [0, 7],
            'r&g': [6, 7]
            }

models_map = { 'a': "models/models_my_attribute_wise_retrain/adult_a_EIDIG_INF_retrained_model.h5",
            'r': "models/models_my_attribute_wise_retrain/adult_r_EIDIG_INF_retrained_model.h5",
            'g': "models/models_my_attribute_wise_retrain/adult_g_EIDIG_INF_retrained_model.h5",
            'a&r': "models/models_my_attribute_wise_retrain/adult_a&r_EIDIG_INF_retrained_model.h5",
            'a&g': "models/models_my_attribute_wise_retrain/adult_a&g_EIDIG_INF_retrained_model.h5",
            'r&g': "models/models_my_attribute_wise_retrain/adult_r&g_EIDIG_INF_retrained_model.h5",
    
}

for attr in pos_map.keys():
    print("attr", attr)
    target_model_path = models_map[attr]
    data_name = f"discriminatory_data/adult/C-{attr}_ids_EIDIG_INF.npy"
    dis_data = np.load(data_name)
    num_attribs = len(X_train[0])
    protected_attribs = pos_map[attr]
    similar_X = similar_set(dis_data, num_attribs, protected_attribs, constraint)

    augmented_model = keras.models.load_model(target_model_path)
    aug_data = (augmented_model.predict(dis_data) > 0.5).astype(int).flatten()
    dis_num = 0
    newdata_res = []

    l = len(similar_X)
    for i in range(l):
        newdata_re = augmented_model.predict(similar_X[i])
        newdata_re = (newdata_re > 0.5).astype(int).flatten()
        newdata_res.append(newdata_re)
    repaired_num = get_repaired_num(newdata_res)

    print('Aug', repaired_num/len(dis_data))

In [ ]:
X_train, X_val, y_train, y_val, constraint = pre_census_income.X_train, \
    pre_census_income.X_val, pre_census_income.y_train, pre_census_income.y_val, pre_census_income.constraint
    
X_test, y_test = pre_census_income.X_test, pre_census_income.y_test

pos_map = { 'a': [0],
            'r': [6],
            'g': [7],
            'a&r': [0, 6],
            'a&g': [0, 7],
            'r&g': [6, 7]
            }

models_map = { 'a': "models/models_flipped_retrain/adult_a_flipped_retrained_model.h5",
            'r': "models/models_flipped_retrain/adult_r_flipped_retrained_model.h5,,
            'g': "models/models_flipped_retrain/adult_g_flipped_retrained_model.h5",
            'a&r': "models/models_flipped_retrain/adult_a&r_flipped_retrained_model.h5",
            'a&g': "models/models_flipped_retrain/adult_a&g_flipped_retrained_model.h5",
            'r&g': "models/models_flipped_retrain/adult_r&g_flipped_retrained_model.h5",
    
}

for attr in pos_map.keys():
    print("attr", attr)
    target_model_path = models_map[attr]
    data_name = f"discriminatory_data/adult/C-{attr}_ids_EIDIG_INF.npy"
    dis_data = np.load(data_name)
    num_attribs = len(X_train[0])
    protected_attribs = pos_map[attr]
    similar_X = similar_set(dis_data, num_attribs, protected_attribs, constraint)

    augmented_model = keras.models.load_model(target_model_path)
    aug_data = (augmented_model.predict(dis_data) > 0.5).astype(int).flatten()
    dis_num = 0
    newdata_res = []

    l = len(similar_X)
    for i in range(l):
        newdata_re = augmented_model.predict(similar_X[i])
        newdata_re = (newdata_re > 0.5).astype(int).flatten()
        newdata_res.append(newdata_re)
    repaired_num = get_repaired_num(newdata_res)

    print('Aug', repaired_num/len(dis_data))